In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("data/DLS_grid.csv", encoding = "ISO-8859-1") # Read csv file
df.replace("--", np.nan, inplace=True) # Replace -- with NaN
df.replace("Multimodal", np.nan, inplace=True) # Replace Multimodal with NaN
# Change object types to float in relevant columns
df[["Radius (nm)", "%PD", "Range1 Radius (I) (0.1-10nm)", "Range1 %Pd (I)"]] = df[["Radius (nm)","%PD", "Range1 Radius (I) (0.1-10nm)", "Range1 %Pd (I)"]].astype(float)

C:\Users\BEIM\AppData\Local\Temp\ipykernel_39164\3210721078.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("--", np.nan, inplace=True) # Replace -- with NaN


In [3]:
def split_by_temp(dataframe: pd.DataFrame,
                  column: str, 
                  column_temperature = 'Set Temp (C)', 
                  column_well = 'Well', 
                  start_temperature = 25.0
                  ) -> pd.DataFrame:
    """
    Takes an input data frame and the name of the column that should be split, and divides
    this column into a 2d data frame with the Well IDs as the first column, and each temperature 
    the experiment was ran at in subsequent columns.

    Inputs: 
        dataframe 
            type: pd.DataFrame 
            description: dataframe with a format generated by Wyatt Dynamics DLS software.
        column 
            type: str
            description: string referencing the name of the column in dataframe where the 
                         desired output values are stored.
        column_temperature
            type: str
            default value: 'Set Temp (C)'
            description: Name of the column containing the set temperature data.
        column_well
            type: str
            default value: 'Well'
            description: Name of the column containing the well information in the dataframe.
        start_temperature
            type: float
            default value: 25.0
            description: First temperature at which the experiment was run at. See the first 
                         value in the set temperature column if unsure.
    
    Output:
        type: pd.DataFrame

    Example:
        radius = split_by_temp(df, 'Radius (nm)')
        print(radius)

        >      Well      25C      35C       45C      55C       65C        75C      85C
        >   0    A1     2.21     2.77      2.75     2.78      3.02       3.11     5.32
        >   1    A2  1796.85  1105.19  15546.31  9875.37  11783.51   16179.94  8815.53
        >   2    A3     1.95     2.00      2.98     2.92      2.88       3.27    39.60
        >   3    A4     2.59     2.49      2.93     2.41      3.21       3.97     4.03
        >   4    A5     1.99     2.72      2.66     2.89      3.34      10.42     4.04
        >   ..  ...      ...      ...       ...      ...       ...        ...      ...
        >   91   H8     3.40     3.29      3.08     3.11      3.52     472.80      NaN
        >   92   H9     1.58     2.28      2.38     2.43      4.18     512.16      NaN
        >   93  H10   482.33   849.22   2085.64  2497.46      2.17  266799.55      NaN
        >   94  H11     1.92     2.53     27.36     2.75      1.16     197.88      NaN
        >   95  H12    74.02    86.16      9.69     1.48      1.42     678.38      NaN
        >
        >   [96 rows x 8 columns]
    """
    # Create framework for output dataframe with well numbers
    output = dataframe.loc[dataframe[column_temperature] == start_temperature, [column_well]]

    for index, row in dataframe.iterrows(): # Iterate each row in dataframe

        if row[column_well] == dataframe.at[0,column_well]: # Skip all rows that are not referencing the first well
            set_temperature = row[column_temperature] # Set current temperature
            new_column_name = str(set_temperature).split(sep='.')[0] + 'C' # Re-format set temperature to desired column name

            output[new_column_name] = dataframe.loc[ # create new column for set temperature in output
                dataframe[column_temperature] == set_temperature, # find values in dataframe where the temperature == set temperature
                [column] # find all values of interest corresponding to the boolean check above
                ].values # only extract the values as a numpy array
    return output 

In [4]:
radius = split_by_temp(df, 'Radius (nm)')
radius

,Well,25C,35C,45C,55C,65C,75C,85C
0,A1,2.21,2.77,2.75,2.78,3.02,3.11,5.32
1,A2,1796.85,1105.19,15546.31,9875.37,11783.51,16179.94,8815.53
2,A3,1.95,2.00,2.98,2.92,2.88,3.27,39.60
3,A4,2.59,2.49,2.93,2.41,3.21,3.97,4.03
4,A5,1.99,2.72,2.66,2.89,3.34,10.42,4.04
...,...,...,...,...,...,...,...,...
91,H8,3.40,3.29,3.08,3.11,3.52,472.80,NaN
92,H9,1.58,2.28,2.38,2.43,4.18,512.16,NaN
93,H10,482.33,849.22,2085.64,2497.46,2.17,266799.55,NaN
94,H11,1.92,2.53,27.36,2.75,1.16,197.88,NaN


In [5]:
polydispersity = split_by_temp(df, '%PD')
polydispersity

,Well,25C,35C,45C,55C,65C,75C,85C
0,A1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A2,12.1,47.8,NaN,NaN,0.0,0.0,NaN
2,A3,27.5,37.5,NaN,NaN,NaN,NaN,NaN
3,A4,NaN,55.9,NaN,NaN,NaN,NaN,NaN
4,A5,33.3,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
91,H8,32.1,22.8,26.0,NaN,NaN,NaN,NaN
92,H9,34.8,NaN,NaN,NaN,NaN,0.0,NaN
93,H10,NaN,NaN,NaN,NaN,NaN,0.0,NaN
94,H11,33.0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
radius_range1 = split_by_temp(df, 'Range1 Radius (I) (0.1-10nm)')
radius_range1

,Well,25C,35C,45C,55C,65C,75C,85C
0,A1,1.95,1.63,1.54,1.49,1.61,1.66,3.75
1,A2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A3,2.45,2.22,1.68,1.64,1.56,1.71,3.22
3,A4,2.60,2.60,1.92,1.98,2.00,2.71,4.07
4,A5,2.10,1.57,1.54,1.49,1.67,2.35,3.51
...,...,...,...,...,...,...,...,...
91,H8,3.83,4.31,3.96,2.71,3.08,NaN,NaN
92,H9,1.82,1.74,1.43,1.29,1.52,NaN,NaN
93,H10,1.52,1.65,1.46,1.38,1.46,NaN,NaN
94,H11,2.08,1.63,1.74,1.58,1.59,NaN,NaN


In [7]:
PD_radius1 = split_by_temp(df, 'Range1 %Pd (I)')
PD_radius1

,Well,25C,35C,45C,55C,65C,75C,85C
0,A1,10.82,30.99,24.61,22.68,22.45,24.78,16.38
1,A2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A3,29.01,17.64,27.77,31.10,31.87,33.24,60.81
3,A4,12.91,22.49,33.41,17.18,25.37,14.68,22.20
4,A5,23.41,35.44,25.37,26.54,30.02,22.70,23.37
...,...,...,...,...,...,...,...,...
91,H8,34.54,46.20,46.15,34.88,42.01,NaN,NaN
92,H9,32.40,29.54,34.48,19.55,48.95,NaN,NaN
93,H10,49.33,51.27,32.42,32.00,39.82,NaN,NaN
94,H11,22.44,30.86,52.98,39.05,34.56,NaN,NaN


In [8]:
# Export dataframes to excel with each dataframe as a seprate sheet
with pd.ExcelWriter('results/output.xlsx') as writer:
    radius.to_excel(writer, sheet_name='Radius', index=False)
    polydispersity.to_excel(writer, sheet_name='%PD', index=False)
    radius_range1.to_excel(writer, sheet_name='Range1 Radius (I) (0.1-10nm)', index=False)
    PD_radius1.to_excel(writer, sheet_name='Range1 %Pd (I)', index=False)